In [2]:
%load_ext autoreload
%autoreload 2

# For debugging turn on logging to console
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

# mongodb
import pymongo as pm

# fine-tuned newspaper lib
from resources.newspaper import newspaper
from resources.newspaper.newspaper.source import Source
from resources.newspaper.newspaper.article import Article

import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser
from time import sleep
import random
import pytz
import datetime

import requests

In [42]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
# DB_NAME = 'scrape'
DB_NAME = 'FINALP'
db = client[DB_NAME]

def saveToDB(db, collection, url, html, meta={}):
    """
    Saves a document to mongoDB, making sure there are no duplicates by 
    'url' value
    
    Parameters:
    --------
    db, collection  : mongo db connection
    url, html, meta : values to store
    
    Returns:
    --------
    Saved document
    """
    collection = db[collection]
    collection.update_one(
        {'url' : url},
        {
            '$set':
                {'url' : url,
                 'html' : html,
                 'meta' : meta
                }
        }
        ,
        upsert=True
    )
    log.debug(f'Saved to DB')

def scrape(url, db, collection):
    '''
    Scrapes an article from the 'url' up to the 'latest_date'
    
    Parameters:
    --------
    url         : main news website url
    date        : YYYY-MM-DD
    db          : database name
    collection  : mongodb collection
    
    Returns:
    --------
    Article's html and features stored to db, 
    Article's publish date
    
    '''
    log.debug(f"Exctracting features from {url}")
    try:
        article = Article(url)
        article.download()
        # the below method may only extract a snippet... 
        # check the database for results of text extraction
        # and apply additional processing if needed after 
        # article has been stored in the DB
        # see code below Newrepublic for example
        article.parse()
        soup = bs.BeautifulSoup(article.html, 'lxml')
        article.publish_date = date_parser(soup.find('span', {'class':'dateline'}).text)
        
    except Exception as e:
        log.critical(f'Data not saved: {e}')
        return datetime.datetime.now()
    
    saveToDB(db, collection, article.url, article.html, meta={
        'date'    :article.publish_date,
        'title'   :article.title,
        'text'    :article.text,
        'authors' :article.authors
    })
    
    return article.publish_date

def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [49]:
collection = 'mjones'
source = 'https://www.motherjones.com/politics/page/'
base_url = 'https://www.motherjones.com/'
page   = 1

utc=pytz.UTC
earliest_date = date_parser('2017-01-01')
scraped_url = []

while True:
    log.debug(f'\n\n PROCESSING PAGE: {source+str(page)}\n\n\
              ====================================\n\n')
    r = requests.get(source+str(page))
       
    page += 1
    soup = bs.BeautifulSoup(r.text,'lxml')

    for link in soup.find('ul', {'class':'articles-list'}).findChildren("a" , recursive=True):
        url = link['href']
        
        if url and url not in scraped_url and base_url in url:
            scraped_url.append(url)
            log.debug(f'Processing url: {url}')
        
            article_date = scrape(url, db, collection)
        
            
    if article_date < earliest_date:
        log.debug(f'Reached earliest date requested: {article_date}')
        break

DEBUG:__main__:

 PROCESSING PAGE: https://www.motherjones.com/politics/page/1



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/page/1 HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/ HTTP/1.1" 200 34790
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2018/07/the-administration-had-no-plan-to-prevent-widespread-family-separations/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2018/07/the-administration-had-no-plan-to-prevent-widespread-family-separations/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2018/07/the-administration-had-no-plan-to-prevent-widespread-family-separations/ HTTP/1.1" 200 34964
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www

DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /author/pema-levy/ HTTP/1.1" 200 34854
CRITICAL:__main__:Data not saved: 'NoneType' object has no attribute 'text'
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2018/07/dana-rohrabacher-says-anyone-in-this-town-would-meet-with-russians-peddling-political-dirt/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2018/07/dana-rohrabacher-says-anyone-in-this-town-would-meet-with-russians-peddling-political-dirt/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2018/07/dana-rohrabacher-says-anyone-in-this-town-would-meet-with-russians-peddling-political-dirt/ HTTP/1.1" 200 36661
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/author/david-corn/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/author/david-corn/


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2018/07/rick-scott-is-getting-a-boost-from-a-private-prison-company-too-bad-it-might-be-illegal/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/author/samantha-michaels/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/author/samantha-michaels/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /author/samantha-michaels/ HTTP/1.1" 200 34832
CRITICAL:__main__:Data not saved: 'NoneType' object has no attribute 'text'
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2018/07/michael-cohen-donald-trump-trump-tower-collusion/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2018/07/michael-cohen-donald-trump-trump

DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/page/3 HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/page/3/ HTTP/1.1" 200 35115
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2018/07/brian-kemp-casey-cagle-georgia-runoff/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2018/07/brian-kemp-casey-cagle-georgia-runoff/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2018/07/brian-kemp-casey-cagle-georgia-runoff/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/author/jamilah-king/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/author/jamilah-king/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjon

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2018/07/more-than-400-parents-separated-from-their-children-were-potentially-deported-trump-administration-claims/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2018/07/more-than-400-parents-separated-from-their-children-were-potentially-deported-trump-administration-claims/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2018/07/more-than-400-parents-separated-from-their-children-were-potentially-deported-trump-administration-claims/ HTTP/1.1" 200 34653
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2018/07/trump-security-clearances-former-obama-officials-sarah-huckaee-sanders-russia/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2018/07/trump-security-clearances-former-o

...

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2017/01/make-birth-control-expensive-again/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2017/01/make-birth-control-expensive-again/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2017/01/make-birth-control-expensive-again/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2017/01/yet-another-state-takes-aim-new-abortion-ban/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2017/01/yet-another-state-takes-aim-new-abortion-ban/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2017/01/yet-another-state-takes-aim-new-abortion-ban/ HTTP/1.1" 200 None
DE

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2017/01/racists-anonymous-church-national-discussion-race/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2017/01/obama-farewell-highlights-chicago-speech-video/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2017/01/obama-farewell-highlights-chicago-speech-video/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2017/01/obama-farewell-highlights-chicago-speech-video/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2017/01/president-obama-farewell-speech-watch/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2017/01/president-obam

DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2017/01/jeff-sessions-trump-private-prisons-attorney-general/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2017/01/jeff-sessions-trump-private-prisons-attorney-general/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:

 PROCESSING PAGE: https://www.motherjones.com/politics/page/176



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/page/176 HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/page/176/ HTTP/1.1" 200 None
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2017/01/democrats-turn-pressure-republicans-russian-hacking-investigation/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politi

DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2016/12/trump-usda-cabinet/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2016/12/trump-usda-cabinet/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2016/12/trump-usda-cabinet/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2016/12/read-report-malicious-russian-cyberactivity-interfered-us-election/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2016/12/read-report-malicious-russian-cyberactivity-interfered-us-election/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2016/12/read-report-malicious-russian-cyberactivity-interfered-us-election/ HTTP/1.1" 200 None
DEBUG:__mai

DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2016/12/america-income-inequality-wealth-net-worth-charts/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2016/12/heres-what-corey-lewandowski-said-about-lobbyists-before-becoming-one/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2016/12/heres-what-corey-lewandowski-said-about-lobbyists-before-becoming-one/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.motherjones.com
DEBUG:urllib3.connectionpool:https://www.motherjones.com:443 "GET /politics/2016/12/heres-what-corey-lewandowski-said-about-lobbyists-before-becoming-one/ HTTP/1.1" 200 None
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.motherjones.com/politics/2016/12/virginia-gop-20-week-abortion-ban/
DEBUG:__main__:Exctracting features from https://www.motherjones.com/politics/2016/12/virginia-gop-20-week-abortion-ban